# After running flame for paired t-test analysis, finish mixed fx workflow

In [2]:
import os
import os.path as op

import numpy as np
import pandas as pd
import nibabel as nib
from scipy.ndimage import binary_dilation
from skimage import morphology
import matplotlib as mpl
import matplotlib.pyplot as plt

from nipype import IdentityInterface, Node, Workflow
from nipype.interfaces.base import (BaseInterface,
                                    BaseInterfaceInputSpec,
                                    InputMultiPath, OutputMultiPath,
                                    TraitedSpec, File, traits)
from nipype.interfaces import fsl, freesurfer

import seaborn as sns
from moss import locator
from moss.mosaic import Mosaic

import lyman
from lyman import tools
from lyman.tools import SaveParameters, add_suffix, nii_to_png


/Users/steph-backup/anaconda/lib/python2.7/site-packages/matplotlib/__init__.py:872: UserWarning: axes.color_cycle is deprecated and replaced with axes.prop_cycle; please use the latter.
  warnings.warn(self.msg_depr % (key, alt_key))


### AP: stress group safe vs. threat SH > CR

In [29]:
os.environ["SUBJECTS_DIR"] = '/Volumes/group/awagner/sgagnon/AP/data'
os.environ["LYMAN_DIR"] = '/Volumes/group/awagner/sgagnon/AP/scripts'
exp = 'ap_memory_raw'
altmodel = 'byshockCond'
subjects = '/Volumes/group/awagner/sgagnon/AP/scripts/subjects_shock_byshockCondSH.txt'
basedir = '/Volumes/group/awagner/sgagnon/AP/analysis/ap_memory_raw-byshockCond/group_stress/safe-threat_sourcehit-cr'

project = lyman.gather_project_info()
exp_info = lyman.gather_experiment_info(exp, altmodel)

subject_list = lyman.determine_subjects([subjects])
subj_source = Node(IdentityInterface(fields=["subject_id"]),
                   name="subj_source")
subj_source.inputs.subject_id = subject_list

for contrast_num in [1, 2]:
    
    # Estimate smoothness
    smoothest = fsl.SmoothEstimate()
    smoothest.inputs.mask_file = basedir + '/stats+/mask.nii.gz'
    smoothest.inputs.zstat_file = basedir + '/stats+/zstat'+str(contrast_num)+'.nii.gz'
    smooth = smoothest.run()
    
    # Cluster correct
    cluster = fsl.Cluster(in_file=smoothest.inputs.zstat_file,
                          threshold=exp_info["cluster_zthresh"],
                          pthreshold=exp_info["grf_pthresh"],
                          out_threshold_file=basedir+'/zstat'+str(contrast_num)+'_threshold.nii.gz',
                          out_index_file=basedir+'/zstat'+str(contrast_num)+'_index.nii.gz',
                          out_localmax_txt_file=basedir+'/zstat'+str(contrast_num)+'_localmax.txt',
                          peak_distance=exp_info["peak_distance"],
                          use_mm=True,
                          dlh=smooth.outputs.dlh,
                          volume=smooth.outputs.volume)
    cluster.run()

## SST: Control planning (probe - habit)

In [5]:
os.environ["SUBJECTS_DIR"] = '/Volumes/group/awagner/sgagnon/SST/data'
os.environ["LYMAN_DIR"] = '/Volumes/group/awagner/sgagnon/SST/scripts'
exp = 'mvpa'
altmodel = 'goodruntype_filt'
subjects = '/Volumes/group/awagner/sgagnon/SST/scripts/subjects_control.txt'
basedir = '/Volumes/group/awagner/sgagnon/SST/analysis/mvpa-goodruntype_filt/group_control/probe-habit'

project = lyman.gather_project_info()
exp_info = lyman.gather_experiment_info(exp, altmodel)

subject_list = lyman.determine_subjects([subjects])
subj_source = Node(IdentityInterface(fields=["subject_id"]),
                   name="subj_source")
subj_source.inputs.subject_id = subject_list

for contrast_num in [1, 2]:
    
    # Estimate smoothness
    smoothest = fsl.SmoothEstimate()
    smoothest.inputs.mask_file = basedir + '/stats/mask.nii.gz'
    smoothest.inputs.zstat_file = basedir + '/stats/zstat'+str(contrast_num)+'.nii.gz'
    smooth = smoothest.run()
    
    # Cluster correct
    cluster = fsl.Cluster(in_file=smoothest.inputs.zstat_file,
                          threshold=exp_info["cluster_zthresh"],
                          pthreshold=exp_info["grf_pthresh"],
                          out_threshold_file=basedir+'/zstat'+str(contrast_num)+'_threshold.nii.gz',
                          out_index_file=basedir+'/zstat'+str(contrast_num)+'_index.nii.gz',
                          out_localmax_txt_file=basedir+'/zstat'+str(contrast_num)+'_localmax.txt',
                          peak_distance=exp_info["peak_distance"],
                          use_mm=True,
                          dlh=smooth.outputs.dlh,
                          volume=smooth.outputs.volume)
    cluster.run()

INFO:interface:stdout 2017-08-02T16:16:28.661851:DLH 0.00709493
INFO:interface:stdout 2017-08-02T16:16:28.661851:VOLUME 208553
INFO:interface:stdout 2017-08-02T16:16:28.661851:RESELS 650.699
/Users/steph-backup/anaconda/lib/python2.7/site-packages/nipype-0.11.0-py2.7.egg/nipype/interfaces/base.py:419: UserWarning: Input pthreshold requires inputs: dlh, volume
  warn(msg)
INFO:interface:stdout 2017-08-02T16:16:30.679957:Cluster Index	Voxels	P	-log10(P)	MAX	MAX X (vox)	MAX Y (vox)	MAX Z (vox)	COG X (vox)	COG Y (vox)	COG Z (vox)
INFO:interface:stdout 2017-08-02T16:16:30.679957:1	59823	1.77e-23	22.8	6	60	76	36	47.3	54.6	47.9
INFO:interface:stdout 2017-08-02T16:16:32.174147:DLH 0.00709493
INFO:interface:stdout 2017-08-02T16:16:32.174147:VOLUME 208553
INFO:interface:stdout 2017-08-02T16:16:32.174147:RESELS 650.699
/Users/steph-backup/anaconda/lib/python2.7/site-packages/nipype-0.11.0-py2.7.egg/nipype/interfaces/base.py:419: UserWarning: Input pthreshold requires inputs: dlh, volume
  warn(ms

## SST: Control nav (probe - habit)

In [7]:
os.environ["SUBJECTS_DIR"] = '/Volumes/group/awagner/sgagnon/SST/data'
os.environ["LYMAN_DIR"] = '/Volumes/group/awagner/sgagnon/SST/scripts'
exp = 'mvpa'
altmodel = 'goodruntype_filt'
subjects = '/Volumes/group/awagner/sgagnon/SST/scripts/subjects_control.txt'
basedir = '/Volumes/group/awagner/sgagnon/SST/analysis/mvpa-goodruntype_filt/group_control/nav_probe-habit'

project = lyman.gather_project_info()
exp_info = lyman.gather_experiment_info(exp, altmodel)

subject_list = lyman.determine_subjects([subjects])
subj_source = Node(IdentityInterface(fields=["subject_id"]),
                   name="subj_source")
subj_source.inputs.subject_id = subject_list

for contrast_num in [1, 2]:
    
    # Estimate smoothness
    smoothest = fsl.SmoothEstimate()
    smoothest.inputs.mask_file = basedir + '/stats/mask.nii.gz'
    smoothest.inputs.zstat_file = basedir + '/stats/zstat'+str(contrast_num)+'.nii.gz'
    smooth = smoothest.run()
    
    # Cluster correct
    cluster = fsl.Cluster(in_file=smoothest.inputs.zstat_file,
                          threshold=exp_info["cluster_zthresh"],
                          pthreshold=exp_info["grf_pthresh"],
                          out_threshold_file=basedir+'/zstat'+str(contrast_num)+'_threshold.nii.gz',
                          out_index_file=basedir+'/zstat'+str(contrast_num)+'_index.nii.gz',
                          out_localmax_txt_file=basedir+'/zstat'+str(contrast_num)+'_localmax.txt',
                          peak_distance=exp_info["peak_distance"],
                          use_mm=True,
                          dlh=smooth.outputs.dlh,
                          volume=smooth.outputs.volume)
    cluster.run()

INFO:interface:stdout 2017-08-02T16:47:26.715750:DLH 0.0126696
INFO:interface:stdout 2017-08-02T16:47:26.715750:VOLUME 208553
INFO:interface:stdout 2017-08-02T16:47:26.715750:RESELS 364.389
/Users/steph-backup/anaconda/lib/python2.7/site-packages/nipype-0.11.0-py2.7.egg/nipype/interfaces/base.py:419: UserWarning: Input pthreshold requires inputs: dlh, volume
  warn(msg)
INFO:interface:stdout 2017-08-02T16:47:27.852806:Cluster Index	Voxels	P	-log10(P)	MAX	MAX X (vox)	MAX Y (vox)	MAX Z (vox)	COG X (vox)	COG Y (vox)	COG Z (vox)
INFO:interface:stdout 2017-08-02T16:47:27.852806:1	8124	1.12e-08	7.95	4.99	41	82	52	30.6	77.8	49.8
INFO:interface:stdout 2017-08-02T16:47:29.492612:DLH 0.0126696
INFO:interface:stdout 2017-08-02T16:47:29.492612:VOLUME 208553
INFO:interface:stdout 2017-08-02T16:47:29.492612:RESELS 364.389
/Users/steph-backup/anaconda/lib/python2.7/site-packages/nipype-0.11.0-py2.7.egg/nipype/interfaces/base.py:419: UserWarning: Input pthreshold requires inputs: dlh, volume
  warn(ms

## SST: Stress planning (probe - habit)

In [9]:
os.environ["SUBJECTS_DIR"] = '/Volumes/group/awagner/sgagnon/SST/data'
os.environ["LYMAN_DIR"] = '/Volumes/group/awagner/sgagnon/SST/scripts'
exp = 'mvpa'
altmodel = 'goodruntype_filt'
subjects = '/Volumes/group/awagner/sgagnon/SST/scripts/subjects_stress.txt'
basedir = '/Volumes/group/awagner/sgagnon/SST/analysis/mvpa-goodruntype_filt/group_stress/probe-habit'

project = lyman.gather_project_info()
exp_info = lyman.gather_experiment_info(exp, altmodel)

subject_list = lyman.determine_subjects([subjects])
subj_source = Node(IdentityInterface(fields=["subject_id"]),
                   name="subj_source")
subj_source.inputs.subject_id = subject_list

for contrast_num in [1, 2]:
    
    # Estimate smoothness
    smoothest = fsl.SmoothEstimate()
    smoothest.inputs.mask_file = basedir + '/stats/mask.nii.gz'
    smoothest.inputs.zstat_file = basedir + '/stats/zstat'+str(contrast_num)+'.nii.gz'
    smooth = smoothest.run()
    
    # Cluster correct
    cluster = fsl.Cluster(in_file=smoothest.inputs.zstat_file,
                          threshold=exp_info["cluster_zthresh"],
                          pthreshold=exp_info["grf_pthresh"],
                          out_threshold_file=basedir+'/zstat'+str(contrast_num)+'_threshold.nii.gz',
                          out_index_file=basedir+'/zstat'+str(contrast_num)+'_index.nii.gz',
                          out_localmax_txt_file=basedir+'/zstat'+str(contrast_num)+'_localmax.txt',
                          peak_distance=exp_info["peak_distance"],
                          use_mm=True,
                          dlh=smooth.outputs.dlh,
                          volume=smooth.outputs.volume)
    cluster.run()

INFO:interface:stdout 2017-08-02T17:37:35.439309:DLH 0.0125923
INFO:interface:stdout 2017-08-02T17:37:35.439309:VOLUME 213456
INFO:interface:stdout 2017-08-02T17:37:35.439309:RESELS 366.626
/Users/steph-backup/anaconda/lib/python2.7/site-packages/nipype-0.11.0-py2.7.egg/nipype/interfaces/base.py:419: UserWarning: Input pthreshold requires inputs: dlh, volume
  warn(msg)
INFO:interface:stdout 2017-08-02T17:37:36.419013:Cluster Index	Voxels	P	-log10(P)	MAX	MAX X (vox)	MAX Y (vox)	MAX Z (vox)	COG X (vox)	COG Y (vox)	COG Z (vox)
INFO:interface:stdout 2017-08-02T17:37:36.419013:3	10490	2.41e-10	9.62	5.33	36	32	49	44.3	31	55.9
INFO:interface:stdout 2017-08-02T17:37:36.419013:2	5132	3.4e-06	5.47	4.85	64	75	48	64.9	65.7	49.3
INFO:interface:stdout 2017-08-02T17:37:36.419013:1	3931	4.3e-05	4.37	4.57	21	75	49	33.5	71.3	58.4
INFO:interface:stdout 2017-08-02T17:37:37.766138:DLH 0.0125923
INFO:interface:stdout 2017-08-02T17:37:37.766138:VOLUME 213456
INFO:interface:stdout 2017-08-02T17:37:37.766138:

## SST: Stress nav (probe - habit)

In [8]:
os.environ["SUBJECTS_DIR"] = '/Volumes/group/awagner/sgagnon/SST/data'
os.environ["LYMAN_DIR"] = '/Volumes/group/awagner/sgagnon/SST/scripts'
exp = 'mvpa'
altmodel = 'goodruntype_filt'
subjects = '/Volumes/group/awagner/sgagnon/SST/scripts/subjects_stress.txt'
basedir = '/Volumes/group/awagner/sgagnon/SST/analysis/mvpa-goodruntype_filt/group_stress/nav_probe-habit'

project = lyman.gather_project_info()
exp_info = lyman.gather_experiment_info(exp, altmodel)

subject_list = lyman.determine_subjects([subjects])
subj_source = Node(IdentityInterface(fields=["subject_id"]),
                   name="subj_source")
subj_source.inputs.subject_id = subject_list

for contrast_num in [1, 2]:
    
    # Estimate smoothness
    smoothest = fsl.SmoothEstimate()
    smoothest.inputs.mask_file = basedir + '/stats/mask.nii.gz'
    smoothest.inputs.zstat_file = basedir + '/stats/zstat'+str(contrast_num)+'.nii.gz'
    smooth = smoothest.run()
    
    # Cluster correct
    cluster = fsl.Cluster(in_file=smoothest.inputs.zstat_file,
                          threshold=exp_info["cluster_zthresh"],
                          pthreshold=exp_info["grf_pthresh"],
                          out_threshold_file=basedir+'/zstat'+str(contrast_num)+'_threshold.nii.gz',
                          out_index_file=basedir+'/zstat'+str(contrast_num)+'_index.nii.gz',
                          out_localmax_txt_file=basedir+'/zstat'+str(contrast_num)+'_localmax.txt',
                          peak_distance=exp_info["peak_distance"],
                          use_mm=True,
                          dlh=smooth.outputs.dlh,
                          volume=smooth.outputs.volume)
    cluster.run()

INFO:interface:stdout 2017-08-02T17:18:49.798773:DLH 0.00893559
INFO:interface:stdout 2017-08-02T17:18:49.798773:VOLUME 213456
INFO:interface:stdout 2017-08-02T17:18:49.798773:RESELS 516.66
/Users/steph-backup/anaconda/lib/python2.7/site-packages/nipype-0.11.0-py2.7.egg/nipype/interfaces/base.py:419: UserWarning: Input pthreshold requires inputs: dlh, volume
  warn(msg)
INFO:interface:stdout 2017-08-02T17:18:50.452524:Cluster Index	Voxels	P	-log10(P)	MAX	MAX X (vox)	MAX Y (vox)	MAX Z (vox)	COG X (vox)	COG Y (vox)	COG Z (vox)
INFO:interface:stdout 2017-08-02T17:18:52.134072:DLH 0.00893559
INFO:interface:stdout 2017-08-02T17:18:52.134072:VOLUME 213456
INFO:interface:stdout 2017-08-02T17:18:52.134072:RESELS 516.66
/Users/steph-backup/anaconda/lib/python2.7/site-packages/nipype-0.11.0-py2.7.egg/nipype/interfaces/base.py:419: UserWarning: Input pthreshold requires inputs: dlh, volume
  warn(msg)
INFO:interface:stdout 2017-08-02T17:18:52.509355:Cluster Index	Voxels	P	-log10(P)	MAX	MAX X (vox

In [12]:
os.environ["SUBJECTS_DIR"] = '/Volumes/group/awagner/sgagnon/SST/data'
os.environ["LYMAN_DIR"] = '/Volumes/group/awagner/sgagnon/SST/scripts'
exp = 'mvpa'
altmodel = 'goodruntype_filt'
subjects = '/Volumes/group/awagner/sgagnon/SST/scripts/subjects.txt'
basedir = '/Volumes/group/awagner/sgagnon/SST/analysis/mvpa-goodruntype_filt/group/probe-habit_control-stress'

project = lyman.gather_project_info()
exp_info = lyman.gather_experiment_info(exp, altmodel)

subject_list = lyman.determine_subjects([subjects])
subj_source = Node(IdentityInterface(fields=["subject_id"]),
                   name="subj_source")
subj_source.inputs.subject_id = subject_list

for contrast_num in [1, 2]:
    
    # Estimate smoothness
    smoothest = fsl.SmoothEstimate()
    smoothest.inputs.mask_file = basedir + '/stats/mask.nii.gz'
    smoothest.inputs.zstat_file = basedir + '/stats/zstat'+str(contrast_num)+'.nii.gz'
    smooth = smoothest.run()
    
    # Cluster correct
    cluster = fsl.Cluster(in_file=smoothest.inputs.zstat_file,
                          threshold=exp_info["cluster_zthresh"],
                          pthreshold=exp_info["grf_pthresh"],
                          out_threshold_file=basedir+'/zstat'+str(contrast_num)+'_threshold.nii.gz',
                          out_index_file=basedir+'/zstat'+str(contrast_num)+'_index.nii.gz',
                          out_localmax_txt_file=basedir+'/zstat'+str(contrast_num)+'_localmax.txt',
                          peak_distance=exp_info["peak_distance"],
                          use_mm=True,
                          dlh=smooth.outputs.dlh,
                          volume=smooth.outputs.volume)
    cluster.run()

INFO:interface:stdout 2017-08-03T13:39:52.816864:DLH 0.00768363
INFO:interface:stdout 2017-08-03T13:39:52.816864:VOLUME 208223
INFO:interface:stdout 2017-08-03T13:39:52.816864:RESELS 600.844
/Users/steph-backup/anaconda/lib/python2.7/site-packages/nipype-0.11.0-py2.7.egg/nipype/interfaces/base.py:419: UserWarning: Input pthreshold requires inputs: dlh, volume
  warn(msg)
INFO:interface:stdout 2017-08-03T13:39:54.045282:Cluster Index	Voxels	P	-log10(P)	MAX	MAX X (vox)	MAX Y (vox)	MAX Z (vox)	COG X (vox)	COG Y (vox)	COG Z (vox)
INFO:interface:stdout 2017-08-03T13:39:54.045282:1	59820	9.6e-25	24	7.31	59	76	36	46.8	52.8	48.1
INFO:interface:stdout 2017-08-03T13:39:54.938788:DLH 0.0143179
INFO:interface:stdout 2017-08-03T13:39:54.938788:VOLUME 208223
INFO:interface:stdout 2017-08-03T13:39:54.938788:RESELS 322.44
/Users/steph-backup/anaconda/lib/python2.7/site-packages/nipype-0.11.0-py2.7.egg/nipype/interfaces/base.py:419: UserWarning: Input pthreshold requires inputs: dlh, volume
  warn(msg)

In [14]:
os.environ["SUBJECTS_DIR"] = '/Volumes/group/awagner/sgagnon/SST/data'
os.environ["LYMAN_DIR"] = '/Volumes/group/awagner/sgagnon/SST/scripts'
exp = 'mvpa'
altmodel = 'goodruntype_filt'
subjects = '/Volumes/group/awagner/sgagnon/SST/scripts/subjects.txt'
basedir = '/Volumes/group/awagner/sgagnon/SST/analysis/mvpa-goodruntype_filt/group/probe-habit_control-stress_mumford'

project = lyman.gather_project_info()
exp_info = lyman.gather_experiment_info(exp, altmodel)

subject_list = lyman.determine_subjects([subjects])
subj_source = Node(IdentityInterface(fields=["subject_id"]),
                   name="subj_source")
subj_source.inputs.subject_id = subject_list

for contrast_num in [1, 2, 3]:
    
    # Estimate smoothness
    smoothest = fsl.SmoothEstimate()
    smoothest.inputs.mask_file = basedir + '/stats/mask.nii.gz'
    smoothest.inputs.zstat_file = basedir + '/stats/zstat'+str(contrast_num)+'.nii.gz'
    smooth = smoothest.run()
    
    # Cluster correct
    cluster = fsl.Cluster(in_file=smoothest.inputs.zstat_file,
                          threshold=exp_info["cluster_zthresh"],
                          pthreshold=exp_info["grf_pthresh"],
                          out_threshold_file=basedir+'/zstat'+str(contrast_num)+'_threshold.nii.gz',
                          out_index_file=basedir+'/zstat'+str(contrast_num)+'_index.nii.gz',
                          out_localmax_txt_file=basedir+'/zstat'+str(contrast_num)+'_localmax.txt',
                          peak_distance=exp_info["peak_distance"],
                          use_mm=True,
                          dlh=smooth.outputs.dlh,
                          volume=smooth.outputs.volume)
    cluster.run()

INFO:interface:stdout 2017-08-03T14:28:18.250612:DLH 0.00703817
INFO:interface:stdout 2017-08-03T14:28:18.250612:VOLUME 208223
INFO:interface:stdout 2017-08-03T14:28:18.250612:RESELS 655.947
/Users/steph-backup/anaconda/lib/python2.7/site-packages/nipype-0.11.0-py2.7.egg/nipype/interfaces/base.py:419: UserWarning: Input pthreshold requires inputs: dlh, volume
  warn(msg)
INFO:interface:stdout 2017-08-03T14:28:18.617807:Cluster Index	Voxels	P	-log10(P)	MAX	MAX X (vox)	MAX Y (vox)	MAX Z (vox)	COG X (vox)	COG Y (vox)	COG Z (vox)
INFO:interface:stdout 2017-08-03T14:28:18.617807:1	61891	6.72e-24	23.2	6.37	59	75	36	47.3	54.6	47.5
INFO:interface:stdout 2017-08-03T14:28:20.059803:DLH 0.0117295
INFO:interface:stdout 2017-08-03T14:28:20.059803:VOLUME 208223
INFO:interface:stdout 2017-08-03T14:28:20.059803:RESELS 393.594
/Users/steph-backup/anaconda/lib/python2.7/site-packages/nipype-0.11.0-py2.7.egg/nipype/interfaces/base.py:419: UserWarning: Input pthreshold requires inputs: dlh, volume
  warn(

In [16]:
os.environ["SUBJECTS_DIR"] = '/Volumes/group/awagner/sgagnon/SST/data'
os.environ["LYMAN_DIR"] = '/Volumes/group/awagner/sgagnon/SST/scripts'
exp = 'mvpa'
altmodel = 'goodruntype_filt'
subjects = '/Volumes/group/awagner/sgagnon/SST/scripts/subjects.txt'
basedir = '/Volumes/group/awagner/sgagnon/SST/analysis/mvpa-goodruntype_filt/group/probe-habit_control-stress_mumford_restrictedROI'

project = lyman.gather_project_info()
exp_info = lyman.gather_experiment_info(exp, altmodel)

subject_list = lyman.determine_subjects([subjects])
subj_source = Node(IdentityInterface(fields=["subject_id"]),
                   name="subj_source")
subj_source.inputs.subject_id = subject_list

for contrast_num in [1, 2, 3]:
    
    # Estimate smoothness
    smoothest = fsl.SmoothEstimate()
    smoothest.inputs.mask_file = basedir + '/stats/mask.nii.gz'
    smoothest.inputs.zstat_file = basedir + '/stats/zstat'+str(contrast_num)+'.nii.gz'
    smooth = smoothest.run()
    
    # Cluster correct
    cluster = fsl.Cluster(in_file=smoothest.inputs.zstat_file,
                          threshold=exp_info["cluster_zthresh"],
                          pthreshold=exp_info["grf_pthresh"],
                          out_threshold_file=basedir+'/zstat'+str(contrast_num)+'_threshold.nii.gz',
                          out_index_file=basedir+'/zstat'+str(contrast_num)+'_index.nii.gz',
                          out_localmax_txt_file=basedir+'/zstat'+str(contrast_num)+'_localmax.txt',
                          use_mm=True,
                          dlh=smooth.outputs.dlh,
                          volume=smooth.outputs.volume)
    cluster.run()

INFO:interface:stdout 2017-08-04T13:39:50.869273:DLH 0.00103033
INFO:interface:stdout 2017-08-04T13:39:50.869273:VOLUME 19330
INFO:interface:stdout 2017-08-04T13:39:50.869273:RESELS 4480.76
/Users/steph-backup/anaconda/lib/python2.7/site-packages/nipype-0.11.0-py2.7.egg/nipype/interfaces/base.py:419: UserWarning: Input pthreshold requires inputs: dlh, volume
  warn(msg)
INFO:interface:stdout 2017-08-04T13:39:52.062246:Cluster Index	Voxels	P	-log10(P)	MAX	MAX X (vox)	MAX Y (vox)	MAX Z (vox)	COG X (vox)	COG Y (vox)	COG Z (vox)
INFO:interface:stdout 2017-08-04T13:39:52.062246:5	3509	0.0155	1.81	5.54	66	70	54	63.8	74.4	50.8
INFO:interface:stdout 2017-08-04T13:39:52.062246:4	3375	0.0164	1.78	5.65	23	77	52	25	71.6	55.9
INFO:interface:stdout 2017-08-04T13:39:52.062246:3	1782	0.0352	1.45	5.58	26	27	55	23.8	34.1	57.2
INFO:interface:stdout 2017-08-04T13:39:52.062246:2	1544	0.0401	1.4	5.4	61	33	59	62.8	32.5	58.3
INFO:interface:stdout 2017-08-04T13:39:52.062246:1	1279	0.0467	1.33	4.34	33	91	36	29.

In [17]:
os.environ["SUBJECTS_DIR"] = '/Volumes/group/awagner/sgagnon/SST/data'
os.environ["LYMAN_DIR"] = '/Volumes/group/awagner/sgagnon/SST/scripts'
exp = 'mvpa'
altmodel = 'goodruntype_filt'
subjects = '/Volumes/group/awagner/sgagnon/SST/scripts/subjects.txt'
basedir = '/Volumes/group/awagner/sgagnon/SST/analysis/mvpa-goodruntype_filt/group/probe1v2_control-stress_mumford'

project = lyman.gather_project_info()
exp_info = lyman.gather_experiment_info(exp, altmodel)

subject_list = lyman.determine_subjects([subjects])
subj_source = Node(IdentityInterface(fields=["subject_id"]),
                   name="subj_source")
subj_source.inputs.subject_id = subject_list

for contrast_num in [1, 2, 3]:
    
    # Estimate smoothness
    smoothest = fsl.SmoothEstimate()
    smoothest.inputs.mask_file = basedir + '/stats/mask.nii.gz'
    smoothest.inputs.zstat_file = basedir + '/stats/zstat'+str(contrast_num)+'.nii.gz'
    smooth = smoothest.run()
    
    # Cluster correct
    cluster = fsl.Cluster(in_file=smoothest.inputs.zstat_file,
                          threshold=exp_info["cluster_zthresh"],
                          pthreshold=exp_info["grf_pthresh"],
                          out_threshold_file=basedir+'/zstat'+str(contrast_num)+'_threshold.nii.gz',
                          out_index_file=basedir+'/zstat'+str(contrast_num)+'_index.nii.gz',
                          out_localmax_txt_file=basedir+'/zstat'+str(contrast_num)+'_localmax.txt',
                          use_mm=True,
                          dlh=smooth.outputs.dlh,
                          volume=smooth.outputs.volume)
    cluster.run()

INFO:interface:stdout 2017-08-08T09:59:00.534296:DLH 0.0175147
INFO:interface:stdout 2017-08-08T09:59:00.534296:VOLUME 208256
INFO:interface:stdout 2017-08-08T09:59:00.534296:RESELS 263.587
/Users/steph-backup/anaconda/lib/python2.7/site-packages/nipype-0.11.0-py2.7.egg/nipype/interfaces/base.py:419: UserWarning: Input pthreshold requires inputs: dlh, volume
  warn(msg)
INFO:interface:stdout 2017-08-08T09:59:01.609877:Cluster Index	Voxels	P	-log10(P)	MAX	MAX X (vox)	MAX Y (vox)	MAX Z (vox)	COG X (vox)	COG Y (vox)	COG Z (vox)
INFO:interface:stdout 2017-08-08T09:59:01.609877:4	2996	3.47e-05	4.46	4.65	72	43	39	71.8	47.2	37.7
INFO:interface:stdout 2017-08-08T09:59:01.609877:3	1860	0.00141	2.85	3.72	46	88	27	36.4	88	40.7
INFO:interface:stdout 2017-08-08T09:59:01.609877:2	1716	0.00236	2.63	3.94	71	71	39	64.7	73.8	41.3
INFO:interface:stdout 2017-08-08T09:59:01.609877:1	1631	0.00323	2.49	3.83	58	89	45	52	87.7	49.4
INFO:interface:stdout 2017-08-08T09:59:03.755797:DLH 0.0045144
INFO:interface:st